In [1]:
## Importing libraries
import boto3
import pandas as pd
import numpy as np
from random import sample
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.impute import KNNImputer
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
pd.set_option('display.max_columns', 200)


## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'evan-callaghan-bucket'
bucket = s3.Bucket(bucket_name)

file_key = 'Kaggle-American-Express-Default/amex_sample_submission.csv'
file_key2 = 'Kaggle-American-Express-Default/amex_train_labels.csv'
file_key3 = 'Kaggle-American-Express-Default/amex_train_data.csv'
file_key4 = 'Kaggle-American-Express-Default/amex_test_data.csv'

bucket_object = bucket.Object(file_key)
bucket_object2 = bucket.Object(file_key2)
bucket_object3 = bucket.Object(file_key3)
bucket_object4 = bucket.Object(file_key4)

file_object = bucket_object.get()
file_object2 = bucket_object2.get()
file_object3 = bucket_object3.get()
file_object4 = bucket_object4.get()

file_content_stream = file_object.get('Body')
file_content_stream2 = file_object2.get('Body')
file_content_stream3 = file_object3.get('Body')
file_content_stream4 = file_object4.get('Body')

## Creating data-type dictionary for reading the train data-frame
dtype_dict = {'customer_ID': "object", 'S_2': "object", 'P_2': 'float16', 'D_39': 'float16', 'B_1': 'float16','B_2': 'float16',
              'R_1': 'float16','S_3': 'float16','D_41': 'float16','B_3': 'float16','D_42': 'float16','D_43': 'float16','D_44': 'float16',
              'B_4': 'float16','D_45': 'float16','B_5': 'float16','R_2': 'float16','D_46': 'float16','D_47': 'float16','D_48': 'float16',
              'D_49': 'float16','B_6': 'float16','B_7': 'float16','B_8': 'float16','D_50': 'float16','D_51': 'float16','B_9': 'float16',
              'R_3': 'float16','D_52': 'float16','P_3': 'float16','B_10': 'float16','D_53': 'float16','S_5': 'float16','B_11': 'float16',
              'S_6': 'float16','D_54': 'float16','R_4': 'float16','S_7': 'float16','B_12': 'float16','S_8': 'float16','D_55': 'float16',
              'D_56': 'float16','B_13': 'float16','R_5': 'float16','D_58': 'float16','S_9': 'float16','B_14': 'float16','D_59': 'float16',
              'D_60': 'float16','D_61': 'float16','B_15': 'float16','S_11': 'float16','D_62': 'float16','D_63': 'object','D_64': 'object',
              'D_65': 'float16','B_16': 'float16','B_17': 'float16','B_18': 'float16','B_19': 'float16','D_66': 'float16','B_20': 'float16',
              'D_68': 'float16','S_12': 'float16','R_6': 'float16','S_13': 'float16','B_21': 'float16','D_69': 'float16','B_22': 'float16',
              'D_70': 'float16','D_71': 'float16','D_72': 'float16','S_15': 'float16','B_23': 'float16','D_73': 'float16','P_4': 'float16',
              'D_74': 'float16','D_75': 'float16','D_76': 'float16','B_24': 'float16','R_7': 'float16','D_77': 'float16','B_25': 'float16',
              'B_26': 'float16','D_78': 'float16','D_79': 'float16','R_8': 'float16','R_9': 'float16','S_16': 'float16','D_80': 'float16',
              'R_10': 'float16','R_11': 'float16','B_27': 'float16','D_81': 'float16','D_82': 'float16','S_17': 'float16','R_12': 'float16',
              'B_28': 'float16','R_13': 'float16','D_83': 'float16','R_14': 'float16','R_15': 'float16','D_84': 'float16','R_16': 'float16',
              'B_29': 'float16','B_30': 'float16','S_18': 'float16','D_86': 'float16','D_87': 'float16','R_17': 'float16','R_18': 'float16',
              'D_88': 'float16','B_31': 'int64','S_19': 'float16','R_19': 'float16','B_32': 'float16','S_20': 'float16','R_20': 'float16',
              'R_21': 'float16','B_33': 'float16','D_89': 'float16','R_22': 'float16','R_23': 'float16','D_91': 'float16','D_92': 'float16',
              'D_93': 'float16','D_94': 'float16','R_24': 'float16','R_25': 'float16','D_96': 'float16','S_22': 'float16','S_23': 'float16',
              'S_24': 'float16','S_25': 'float16','S_26': 'float16','D_102': 'float16','D_103': 'float16','D_104': 'float16','D_105': 'float16',
              'D_106': 'float16','D_107': 'float16','B_36': 'float16','B_37': 'float16', 'R_26': 'float16','R_27': 'float16','B_38': 'float16',
              'D_108': 'float16','D_109': 'float16','D_110': 'float16','D_111': 'float16','B_39': 'float16','D_112': 'float16','B_40': 'float16',
              'S_27': 'float16','D_113': 'float16','D_114': 'float16','D_115': 'float16','D_116': 'float16','D_117': 'float16','D_118': 'float16',
              'D_119': 'float16','D_120': 'float16','D_121': 'float16','D_122': 'float16','D_123': 'float16','D_124': 'float16','D_125': 'float16',
              'D_126': 'float16','D_127': 'float16','D_128': 'float16','D_129': 'float16','B_41': 'float16','B_42': 'float16','D_130': 'float16',
              'D_131': 'float16','D_132': 'float16','D_133': 'float16','R_28': 'float16','D_134': 'float16','D_135': 'float16','D_136': 'float16',
              'D_137': 'float16','D_138': 'float16','D_139': 'float16','D_140': 'float16','D_141': 'float16','D_142': 'float16','D_143': 'float16',
              'D_144': 'float16','D_145': 'float16'}

## Reading the data
sample_submission = pd.read_csv(file_content_stream)
train_labels = pd.read_csv(file_content_stream2)
train = pd.read_csv(file_content_stream3, dtype = dtype_dict)
#test = pd.read_csv(file_content_stream4)

In [2]:
## Appending target variables
train = pd.merge(train, train_labels, on = 'customer_ID', how = 'left')
train.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938477,0.001734,0.008728,1.006836,0.009224,0.124023,0.008774,0.004707,NaN,NaN,0.000630,0.080994,0.708984,0.170654,0.006203,0.358643,0.525391,0.255615,NaN,0.063904,0.059418,0.006466,0.148682,1.335938,0.008209,0.001423,0.207275,0.736328,0.096191,NaN,0.023376,0.002768,0.008324,1.001953,0.008301,0.161377,0.148315,0.922852,0.354492,0.151978,0.118103,0.001882,0.158569,0.065735,0.018387,0.063660,0.199585,0.308350,0.016357,0.401611,0.091064,CR,O,0.007126,0.007664,NaN,0.652832,0.008522,NaN,0.004730,6.0,0.271973,0.008362,0.515137,0.002644,0.009010,0.004807,0.008339,0.119385,0.004803,0.108276,0.050873,NaN,0.007553,0.080444,0.069092,NaN,0.004326,0.007561,NaN,0.007729,0.000272,0.001575,0.004238,0.001434,NaN,0.002272,0.004059,0.007122,0.002457,0.002310,0.003532,0.506836,0.008034,1.009766,0.084656,0.003820,0.007042,0.000438,0.006451,0.000830,0.005054,NaN,0.0,0.005722,0.007084,NaN,0.000198,0.008911,NaN,1,0.002537,0.005177,0.006626,0.009705,0.007782,0.002449,1.000977,0.002665,0.007481,0.006893,1.503906,1.005859,0.003569,0.008873,0.003948,0.003647,0.004951,0.894043,0.135620,0.911133,0.974609,0.001244,0.766602,1.008789,1.004883,0.893555,NaN,0.669922,0.009972,0.004570,NaN,1.008789,2.0,NaN,0.004326,NaN,NaN,NaN,1.007812,0.210083,0.676758,0.007874,1.0,0.238281,0.0,4.0,0.232178,0.236206,0.0,0.702148,0.434326,0.003057,0.686523,0.008743,1.0,1.002930,1.007812,1.000000,0.006805,NaN,0.002052,0.005970,NaN,0.004345,0.001534,NaN,NaN,NaN,NaN,NaN,0.002426,0.003706,0.003819,NaN,0.000569,0.000610,0.002674,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936523,0.005775,0.004925,1.000977,0.006153,0.126709,0.000798,0.002714,NaN,NaN,0.002525,0.069397,0.712891,0.113220,0.006207,0.353516,0.521484,0.223389,NaN,0.065247,0.057739,0.001614,0.149780,1.339844,0.008369,0.001984,0.202759,0.720703,0.099792,NaN,0.030594,0.002748,0.002481,1.008789,0.005135,0.140991,0.143555,0.919434,0.326660,0.156250,0.118713,0.001610,0.148438,0.093933,0.013039,0.065491,0.151367,0.265137,0.017685,0.406250,0.086792,CR,O,0.002413,0.007149,NaN,0.646973,0.002237,NaN,0.003880,6.0,0.188965,0.004028,0.509277,0.004192,0.007843,0.001283,0.006523,0.140625,0.000094,0.101013,0.040466,NaN,0.004833,0.081421,0.074158,NaN,0.004204,0.005302,NaN,0.001864,0.000978,0.009895,0.007599,0.000509,NaN,0.009811,0.000126,0.005966,0.000395,0.001327,0.007774,0.500977,0.000761,1.009766,0.081848,0.000347,0.007790,0.004311,0.002333,0.009468,0.003754,NaN,0.0,0.007584,0.006676,NaN,0.001143,0.005905,NaN,1,0.008430,0.008980,0.001854,0.009926,0.005989,0.002247,1.006836,0.002508,0.006828,0.002836,1.503906,1.005859,0.000571,0.000391,0.008354,0.008850,0.003180,0.902344,0.136353,0.919922,0.975586,0.004562,0.786133,1.000000,1.003906,0.906738,NaN,0.668457,0.003922,0.004654,NaN,1.002930,2.0,NaN,0.008705,NaN,NaN,NaN,1.007812,0.184082,0.822266,0.003445,1.0,0.247192,0.0,4.0,0.243530,0.241943,0.0,0.707031,0.430420,0.001306,0.686523,0.000755,1.0,1.008789,1.003906,1.008789,0.004406,NaN,0.001034,0.0048

In [3]:
## Builidng a subset of the training data-frame with 1000 randomly sampled customers
listy = set(train['customer_ID'].unique())
samples = sample(listy, 1000)

## Subsetting the data
train_sample = train[np.isin(train['customer_ID'], samples)]

## Printing the shape of the resulting data-frame
train_sample.shape

(12143, 191)

Ideas:

In [ ]:
# from sklearn.linear_model import LinearRegression
# import matplotlib.pyplot as plt

# ## Creating a subset of the training data
# sub = train_sample[train_sample['customer_ID'] == '04305420dfcc6bae8a5943fde9b19a8beff5456e529a0e6bce15655d9b814329']

# ## Defining the input and target variables
# X = np.asarray(sub['S_2'])
# Y = np.asarray(sub['S_5'])

# ## Cleaning the date variable
# df = pd.DataFrame({'time': X, 'value': Y})
# df.time = pd.to_datetime(df.time)

# ## Fitting a linear regression line
# lm_md = LinearRegression().fit(df.time.values.reshape(-1, 1), df['value'].values.reshape(-1, 1)) 

# ## Extracting the linear model coefficients
# print('Coefficient:', lm_md.coef_)

# ## Predicting on the train set
# y_pred = lm_md.predict(df.time.values.astype(float).reshape(-1, 1))
# df['pred'] = y_pred

# ## Plotting results
# ax = df.plot(x='time', y='value', color='black', style='.', figsize = ([14, 8]))
# df.plot(x='time', y='pred', color='orange', linewidth=3, ax=ax, alpha=0.5)
# ax.set_title('My Title')
# ax.set_xlabel('Date')
# ax.set_ylabel('Value')
# plt.show()

In [ ]:
# train_labels[train_labels['customer_ID'] == '04305420dfcc6bae8a5943fde9b19a8beff5456e529a0e6bce15655d9b814329']

In [ ]:
# ## Defining a list of all customer_IDs
# customers = train_samples['customer_ID'].unique()

# ## Initialzing new column in the train_labels_sample data-frame
# train_labels_sample['Spend_Slope'] = np.nan

# ## Looping through each set of customers:
# for customer in customers:
    
#     ## Subsetting by customer
#     customers_temp = train_sample[train_sample['customer_ID'] == customer]
    
#     ## Spend variable subset
#     spend_temp = customer_temp[['S_2', 'S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_9','S_11', 'S_12', 'S_13', 'S_15', 'S_16', 
#                                 'S_17', 'S_18', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27']]
    
#     ## Defining the input and target variables
#     X = spend_temp['S_2']
#     Y = spend_temp.drop(columns = ['S_2'])
    
#     ## Creating the linear regression model
#     lm_md = LinearRegression().fit(X, Y)
    
#     ## Extracting the slope of the line
#     slope = lm_md.coef_
    
#     ## Creating new variable for spend slope
#     train_labels_sample['Spend_Slope'] = np.where(train_labels_sample['customer_ID'] == customer, slope, train_labels_sample['Spend_Slope'])
    
    

## Idea:
Create subsets on customer_ID and create a simple linear regression model to extract the slope of the line

Useful for change in spending habit, change in balance, and change in paymne behaviours.

## Handling missing values:

In [4]:
train.shape

(5531451, 191)

In [5]:
missing = pd.DataFrame(train.isnull().sum()).T
missing

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0,0,45985,0,0,2016,0,1020544,2016,2016,4740137,1658396,274319,0,2017,0,0,1211699,0,718725,4985917,233,0,22268,3142402,0,0,0,29563,301492,0,4084585,0,0,0,2016,0,1020544,0,0,184803,2990943,49519,0,0,2933643,0,106725,0,598052,6923,0,758161,0,217442,0,2016,3137598,0,2016,4908097,2016,216503,0,0,0,0,194473,2016,94917,0,23708,0,0,5475595,0,21773,0,4908954,0,1,2513912,6923,2016,274319,75939,0,5218918,0,21773,0,0,2016,25687,4058614,0,56,0,0,194473,1,0,29563,0,5150035,2016,0,0,5527586,0,0,5525447,0,0,0,0,0,75,0,2016,29563,0,0,157216,0,0,0,0,0,0,19024,445,18593,12847,634,40655,101548,101548,3021431,4990102,101548,0,56,4922146,128703,2016,5502513,1597,5500117,5500117,5497819,2650,53,1400935,176716,176716,176716,176716,176716,176716,176716,176716,176716,176716,176716,176716,176716,116816,0,101548,101548,690,5459973,101548,101548,4988874,42716,0,5336752,5336752,5336752,5336752,5336752,101548,40632,101548,4587043,101548,40727,101548,0


In [6]:
train_sample.shape

(12143, 191)

In [7]:
missing = pd.DataFrame(train_sample.isnull().sum()).T
missing

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0,0,90,0,0,1,0,2138,1,1,10459,3463,482,0,1,0,0,2585,0,1482,11086,0,0,52,6859,0,0,0,53,596,0,9071,0,0,0,1,0,2138,0,0,349,6465,103,0,0,6139,0,212,0,1229,3,0,1596,0,445,0,1,6762,0,1,10834,1,446,0,0,0,0,425,1,187,0,36,0,0,12061,0,31,0,10624,0,0,5554,3,1,482,156,0,11481,0,31,0,0,1,44,8979,0,0,0,0,425,0,0,53,0,11354,1,0,0,12141,0,0,12135,0,0,0,0,0,0,0,1,53,0,0,298,0,0,0,0,0,0,45,0,44,23,3,86,220,220,6769,11093,220,0,2,10953,262,1,12071,4,12024,12024,12023,4,2,2969,381,381,381,381,381,381,381,381,381,381,381,381,381,251,0,220,220,3,12011,220,220,11088,92,0,11702,11702,11702,11702,11702,220,86,220,10112,220,86,220,0


In [ ]:
## Dropping columns with too many missing values (over 4,000,000 missing)
train = train.drop(columns = ['D_42', 'D_49', 'D_53', 'D_66', 'D_73', 'D_76', 'R_9', 'D_82', 'B_29', 'D_87', 'D_88', 'D_106', 'R_26', 'D_108', 'D_110', 
                             'D_111', 'B_39', 'B_42', 'D_132', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_142'])
train_sample = train_sample.drop(columns = ['D_42', 'D_49', 'D_53', 'D_66', 'D_73', 'D_76', 'R_9', 'D_82', 'B_29', 'D_87', 'D_88', 'D_106', 'R_26', 
                                            'D_108', 'D_110', 'D_111', 'B_39', 'B_42', 'D_132', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_142'])

### KNN Imputer

#### Sample data-frame:

In [8]:
## Defining the input variables and dropping categorical variables
knn_train_sample = train_sample.drop(columns = ['customer_ID', 'S_2', 'D_63', 'D_64', 'target'])

## Building the KNN model
knn_train_sample_fixed = KNNImputer(n_neighbors = 5).fit_transform(knn_train_sample)

## Defining data-frames to store results
DF1 = train_sample[['customer_ID', 'S_2', 'D_63', 'D_64', 'target']].reset_index(drop = True)
DF2 = pd.DataFrame(knn_train_sample_fixed, columns = knn_train_sample.columns)

## Returning final data-frame
results = pd.concat([DF1, DF2], axis = 1)

results.head()

,customer_ID,S_2,D_63,D_64,target,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,002071212ea5e4a9d45f96003c41a784715014736ce63e...,2017-03-18,CO,R,0,0.539062,0.002874,0.522461,0.023590,0.001363,0.154785,0.002762,0.798828,0.174438,0.037170,0.130371,0.181396,0.300049,0.472168,0.004925,0.483398,0.312988,0.934570,0.188721,0.003780,0.399414,0.006451,0.042935,0.008476,0.253906,0.000595,0.079834,0.589355,0.028152,0.060684,0.080566,0.466553,0.001367,1.003906,0.005379,0.106995,0.008812,0.702148,0.747559,0.019708,0.051605,0.009880,0.447510,0.006897,0.117920,0.467529,0.517578,0.905273,0.001151,0.400146,0.020828,0.001375,1.003906,0.943848,0.140137,1.006836,1.0,1.005859,5.0,0.343750,0.005455,0.510254,0.005497,0.001781,0.509277,0.008705,0.003635,0.003639,0.200562,0.364990,0.105124,0.007042,0.222046,0.268311,0.015779,0.004768,0.006332,0.104904,0.210693,0.003756,0.002600,0.504883,0.002708,0.371826,0.007793,0.003521,0.008408,0.008621,0.008476,0.005985,0.503613,1.960938,1.001953,0.183105,0.002087,0.001722,0.002546,0.004204,0.003790,0.005295,0.003301,1.0,0.007225,0.000951,1.0,0.002707,0.006359,0.203345,1.0,0.000830,0.009659,0.000401,0.001354,0.007107,0.005665,0.004677,0.005386,0.001647,0.003166,0.003834,0.005962,0.005558,0.003342,0.004940,0.002615,0.007439,0.973633,0.135010,0.977539,0.971680,0.009178,0.250732,1.004883,0.962402,0.244385,0.183838,0.342773,0.006622,0.523926,0.040833,0.021210,4.0,0.005133,0.003088,0.704980,1.004687,0.354132,1.001953,0.340088,0.364258,0.005859,0.0,0.034515,0.0,1.0,0.033752,0.027191,0.0,0.338867,0.150269,0.006851,0.372803,0.003925,0.0,0.009361,0.002686,0.006153,0.000174,0.011839,1.000000,0.993652,0.270996,0.479248,0.002430,0.488037,0.006634,0.103601,0.004776,0.304846,0.006512,0.006504,0.006939,0.541455,0.001740,0.004597,0.005402
1,002071212ea5e4a9d45f96003c41a784715014736ce63e...,2017-04-18,CO,R,0,0.628418,0.004189,0.612305,0.753906,0.004364,0.132812,0.005981,0.508301,0.096783,0.187256,0.002594,0.385498,0.306396,0.258301,0.000139,0.304199,0.322266,0.427979,0.172852,0.032440,0.396729,0.006115,0.062918,0.003397,0.292480,0.206909,0.075989,0.850098,0.128784,0.035835,0.010849,0.562988,0.001349,1.008789,0.003468,0.086365,0.014244,0.496582,0.628418,0.028900,0.058472,0.000017,0.345703,0.123047,0.144653,0.459473,0.331787,0.883789,0.004009,0.408936,0.179932,0.003036,1.002930,0.514160,0.146606,0.915527,1.0,1.007812,5.0,0.331543,0.005245,0.681641,0.000446,0.000407,0.001848,0.254883,0.003481,0.009041,0.309326,0.375732,0.194986,0.008026,0.145020,0.201050,0.012871,0.004898,0.005581,0.164795,0.245850,0.004688,0.007172,0.500488,0.008766,0.271491,0.008095,0.000386,0.009125,0.009338,0.003355,0.006207,0.504395,0.834473,1.001953,0.188965,0.007988,0.007664,0.009712,0.003799,0.003941,0.502441,0.004183,0.0,0.006893,0.009628,1.0,0.006191,0.007702,0.203345,1.0,0.003428,0.007919,0.003801,0.006355,0.007515,0.004185,0.003641,0.005463,0.002640,0.004620,0.004883,0.003452,0.003296,0.007881,0.008057,0.005268,0.006943,0.991211,0.132935,0.986816,0.972168,0.003593,0.254150,1.006836,0.969727,0.255371,0.174194,0.343018,0.000998,0.621094,0.041962,0.020630,3.0,0.204332,0.0

#### Full train data-frame:

In [ ]:
## Defining the input variables and dropping categorical variables
knn_train = train.drop(columns = ['customer_ID', 'S_2', 'D_63', 'D_64', 'target'])

## Building the KNN model
knn_train_fixed = KNNImputer(n_neighbors = 5).fit_transform(knn_train)

## Defining data-frames to store results
DF1 = train[['customer_ID', 'S_2', 'D_63', 'D_64', 'target']].reset_index(drop = True)
DF2 = pd.DataFrame(knn_train_fixed, columns = knn_train.columns)

## Returning final data-frame
pd.concat([DF1, DF2], axis = 1)

## "Payment" and "Spend" variable engineering:

In [ ]:
p_variables = train.columns
[x for x in p_variables if x.startswith('P_')]

In [ ]:
## Creating a new figure
fig, axes = plt.subplots(2, 2, figsize = (18, 12))

sns.boxplot(ax = axes[0, 0], x = 'target', y = 'P_2', hue = 'target', data = train)
sns.boxplot(ax = axes[0, 1], x = 'target', y = 'P_3', hue = 'target', data = train)
sns.boxplot(ax = axes[1, 0], x = 'target', y = 'P_4', hue = 'target', data = train)

In [ ]:
## Sum of 'P' variables
sum_value = train.groupby('customer_ID')[['P_2', 'P_3', 'P_4']].sum().reset_index(drop = False)
target = train.groupby('customer_ID')['target'].max().reset_index(drop = False)
target_sums = sum_value.merge(target, how = 'left', on = 'customer_ID')
target_sums.columns = ['customer_ID', 'P_2_sum', 'P_3_sum', 'P_4_sum', 'target']

## Creating a new figure
fig, axes = plt.subplots(2, 2, figsize = (18, 12))

sns.boxplot(ax = axes[0, 0], x = 'target', y = 'P_2_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[0, 1], x = 'target', y = 'P_3_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[1, 0], x = 'target', y = 'P_4_sum', hue = 'target', data = target_sums)

### Selections: P_2, P_2_sum, P_3, P_3_sum

In [ ]:
## Grouping the data by customer_ID to obtain the mean, median, and sum P_2 variable values
P_2_mean = pd.DataFrame(train.groupby(['customer_ID'])['P_2'].mean()).reset_index(drop = False)
P_2_median = pd.DataFrame(train.groupby(['customer_ID'])['P_2'].median()).reset_index(drop = False)
P_2_sum = pd.DataFrame(train.groupby(['customer_ID'])['P_2'].sum()).reset_index(drop = False)

## Cleaning the resulting data-frames
P_2_mean.columns = ['customer_ID', 'P_2_mean']
P_2_median.columns = ['customer_ID', 'P_2_median']
P_2_sum.columns = ['customer_ID', 'P_2_sum']

## Merging the resulting data-frames together
P_2_variable = P_2_mean.merge(P_2_median, how = 'left', 
                              on = 'customer_ID').merge(P_2_sum, how = 'left', on = 'customer_ID')

## Merging back into the complete train data-frame
train = train.merge(P_2_variable, how = 'left', on = 'customer_ID')

In [ ]:
## Grouping the data by customer_ID to obtain the mean, median, and sum P_3 variable values
P_3_mean = pd.DataFrame(train.groupby(['customer_ID'])['P_3'].mean()).reset_index(drop = False)
P_3_median = pd.DataFrame(train.groupby(['customer_ID'])['P_3'].median()).reset_index(drop = False)
P_3_sum = pd.DataFrame(train.groupby(['customer_ID'])['P_3'].sum()).reset_index(drop = False)

## Cleaning the resulting data-frames
P_3_mean.columns = ['customer_ID', 'P_3_mean']
P_3_median.columns = ['customer_ID', 'P_3_median']
P_3_sum.columns = ['customer_ID', 'P_3_sum']

## Merging the resulting data-frames together
P_3_variable = P_3_mean.merge(P_3_median, how = 'left', 
                              on = 'customer_ID').merge(P_3_sum, how = 'left', on = 'customer_ID')

## Merging back into the complete train data-frame
train = train.merge(P_3_variable, how = 'left', on = 'customer_ID')

In [ ]:
train.head(13)

In [ ]:
s_variables = train.columns
[x for x in s_variables if x.startswith('S_')]

In [ ]:
## Creating a new figure
fig, axes = plt.subplots(11, 2, figsize = (18, 100))

sns.boxplot(ax = axes[0, 0], x = 'target', y = 'S_3', hue = 'target', data = train)
sns.boxplot(ax = axes[0, 1], x = 'target', y = 'S_5', hue = 'target', data = train)
sns.boxplot(ax = axes[1, 0], x = 'target', y = 'S_6', hue = 'target', data = train)
sns.boxplot(ax = axes[1, 1], x = 'target', y = 'S_7', hue = 'target', data = train)
sns.boxplot(ax = axes[2, 0], x = 'target', y = 'S_8', hue = 'target', data = train)
sns.boxplot(ax = axes[2, 1], x = 'target', y = 'S_9', hue = 'target', data = train)
sns.boxplot(ax = axes[3, 0], x = 'target', y = 'S_11', hue = 'target', data = train)
sns.boxplot(ax = axes[3, 1], x = 'target', y = 'S_12', hue = 'target', data = train)
sns.boxplot(ax = axes[4, 0], x = 'target', y = 'S_13', hue = 'target', data = train)
sns.boxplot(ax = axes[4, 1], x = 'target', y = 'S_15', hue = 'target', data = train)
sns.boxplot(ax = axes[5, 0], x = 'target', y = 'S_16', hue = 'target', data = train)
sns.boxplot(ax = axes[5, 1], x = 'target', y = 'S_17', hue = 'target', data = train)
sns.boxplot(ax = axes[6, 0], x = 'target', y = 'S_18', hue = 'target', data = train)
sns.boxplot(ax = axes[6, 1], x = 'target', y = 'S_19', hue = 'target', data = train)
sns.boxplot(ax = axes[7, 0], x = 'target', y = 'S_20', hue = 'target', data = train)
sns.boxplot(ax = axes[7, 1], x = 'target', y = 'S_22', hue = 'target', data = train)
sns.boxplot(ax = axes[8, 0], x = 'target', y = 'S_23', hue = 'target', data = train)
sns.boxplot(ax = axes[8, 1], x = 'target', y = 'S_24', hue = 'target', data = train)
sns.boxplot(ax = axes[9, 0], x = 'target', y = 'S_25', hue = 'target', data = train)
sns.boxplot(ax = axes[9, 1], x = 'target', y = 'S_26', hue = 'target', data = train)
sns.boxplot(ax = axes[10, 0], x = 'target', y = 'S_27', hue = 'target', data = train)

In [ ]:
## Sum of 'S' variables
sum_value = train.groupby('customer_ID')[['S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_9', 
                                          'S_11', 'S_12', 'S_13', 'S_15', 'S_16', 'S_17', 
                                          'S_18', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 
                                          'S_25', 'S_26', 'S_27']].sum().reset_index(drop = False)
target = train.groupby('customer_ID')['target'].max().reset_index(drop = False)
target_sums = sum_value.merge(target, how = 'left', on = 'customer_ID')
target_sums.columns = ['customer_ID', 'S_3_sum', 'S_5_sum', 'S_6_sum', 'S_7_sum', 'S_8_sum', 'S_9_sum', 
                                          'S_11_sum', 'S_12_sum', 'S_13_sum', 'S_15_sum', 'S_16_sum', 'S_17_sum', 
                                          'S_18_sum', 'S_19_sum', 'S_20_sum', 'S_22_sum', 'S_23_sum', 'S_24_sum', 
                                          'S_25_sum', 'S_26_sum', 'S_27_sum', 'target']

In [ ]:
## Creating a new figure
fig, axes = plt.subplots(11, 2, figsize = (18, 100))

sns.boxplot(ax = axes[0, 0], x = 'target', y = 'S_3_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[0, 1], x = 'target', y = 'S_5_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[1, 0], x = 'target', y = 'S_6_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[1, 1], x = 'target', y = 'S_7_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[2, 0], x = 'target', y = 'S_8_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[2, 1], x = 'target', y = 'S_9_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[3, 0], x = 'target', y = 'S_11_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[3, 1], x = 'target', y = 'S_12_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[4, 0], x = 'target', y = 'S_13_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[4, 1], x = 'target', y = 'S_15_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[5, 0], x = 'target', y = 'S_16_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[5, 1], x = 'target', y = 'S_17_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[6, 0], x = 'target', y = 'S_18_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[6, 1], x = 'target', y = 'S_19_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[7, 0], x = 'target', y = 'S_20_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[7, 1], x = 'target', y = 'S_22_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[8, 0], x = 'target', y = 'S_23_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[8, 1], x = 'target', y = 'S_24_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[9, 0], x = 'target', y = 'S_25_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[9, 1], x = 'target', y = 'S_26_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[10, 0], x = 'target', y = 'S_27_sum', hue = 'target', data = target_sums)

### Selections: S_3, S_3_sum, S_6_sum, S_7, S_7_sum, S_8, S_8_sum, S_13, S_13_sum, S_15, S_27

In [ ]:
## Grouping the data by customer_ID to obtain the mean, median, and sum S_3 variable values
S_3_mean = pd.DataFrame(train.groupby(['customer_ID'])['S_3'].mean()).reset_index(drop = False)
S_3_median = pd.DataFrame(train.groupby(['customer_ID'])['S_3'].median()).reset_index(drop = False)
S_3_sum = pd.DataFrame(train.groupby(['customer_ID'])['S_3'].sum()).reset_index(drop = False)

## Cleaning the resulting data-frames
S_3_mean.columns = ['customer_ID', 'S_3_mean']
S_3_median.columns = ['customer_ID', 'S_3_median']
S_3_sum.columns = ['customer_ID', 'S_3_sum']

## Merging the resulting data-frames together
S_3_variable = S_3_mean.merge(S_3_median, how = 'left', 
                              on = 'customer_ID').merge(S_3_sum, how = 'left', on = 'customer_ID')

## Merging back into the complete train data-frame
train = train.merge(S_3_variable, how = 'left', on = 'customer_ID')

In [ ]:
## Grouping the data by customer_ID to obtain the sum S_6 variable values
S_6_sum = pd.DataFrame(train.groupby(['customer_ID'])['S_6'].sum()).reset_index(drop = False)

## Cleaning the resulting data-frames
S_6_sum.columns = ['customer_ID', 'S_6_sum']

## Merging back into the complete train data-frame
train = train.merge(S_6_sum, how = 'left', on = 'customer_ID')

In [ ]:
## Grouping the data by customer_ID to obtain the mean, median, and sum S_7 variable values
S_7_mean = pd.DataFrame(train.groupby(['customer_ID'])['S_7'].mean()).reset_index(drop = False)
S_7_median = pd.DataFrame(train.groupby(['customer_ID'])['S_7'].median()).reset_index(drop = False)
S_7_sum = pd.DataFrame(train.groupby(['customer_ID'])['S_7'].sum()).reset_index(drop = False)

## Cleaning the resulting data-frames
S_7_mean.columns = ['customer_ID', 'S_7_mean']
S_7_median.columns = ['customer_ID', 'S_7_median']
S_7_sum.columns = ['customer_ID', 'S_7_sum']

## Merging the resulting data-frames together
S_7_variable = S_7_mean.merge(S_7_median, how = 'left', 
                              on = 'customer_ID').merge(S_7_sum, how = 'left', on = 'customer_ID')

## Merging back into the complete train data-frame
train = train.merge(S_7_variable, how = 'left', on = 'customer_ID')

In [ ]:
## Grouping the data by customer_ID to obtain the mean, median, and sum S_8 variable values
S_8_mean = pd.DataFrame(train.groupby(['customer_ID'])['S_8'].mean()).reset_index(drop = False)
S_8_median = pd.DataFrame(train.groupby(['customer_ID'])['S_8'].median()).reset_index(drop = False)
S_8_sum = pd.DataFrame(train.groupby(['customer_ID'])['S_8'].sum()).reset_index(drop = False)

## Cleaning the resulting data-frames
S_8_mean.columns = ['customer_ID', 'S_8_mean']
S_8_median.columns = ['customer_ID', 'S_8_median']
S_8_sum.columns = ['customer_ID', 'S_8_sum']

## Merging the resulting data-frames together
S_8_variable = S_8_mean.merge(S_8_median, how = 'left', 
                              on = 'customer_ID').merge(S_8_sum, how = 'left', on = 'customer_ID')

## Merging back into the complete train data-frame
train = train.merge(S_8_variable, how = 'left', on = 'customer_ID')

In [ ]:
## Grouping the data by customer_ID to obtain the mean, median, and sum S_13 variable values
S_13_mean = pd.DataFrame(train.groupby(['customer_ID'])['S_13'].mean()).reset_index(drop = False)
S_13_median = pd.DataFrame(train.groupby(['customer_ID'])['S_13'].median()).reset_index(drop = False)
S_13_sum = pd.DataFrame(train.groupby(['customer_ID'])['S_13'].sum()).reset_index(drop = False)

## Cleaning the resulting data-frames
S_13_mean.columns = ['customer_ID', 'S_13_mean']
S_13_median.columns = ['customer_ID', 'S_13_median']
S_13_sum.columns = ['customer_ID', 'S_13_sum']

## Merging the resulting data-frames together
S_13_variable = S_13_mean.merge(S_13_median, how = 'left', 
                              on = 'customer_ID').merge(S_13_sum, how = 'left', on = 'customer_ID')

## Merging back into the complete train data-frame
train = train.merge(S_13_variable, how = 'left', on = 'customer_ID')

In [ ]:
## Grouping the data by customer_ID to obtain the mean and median S_15 variable values
S_15_mean = pd.DataFrame(train.groupby(['customer_ID'])['S_15'].mean()).reset_index(drop = False)
S_15_median = pd.DataFrame(train.groupby(['customer_ID'])['S_15'].median()).reset_index(drop = False)

## Cleaning the resulting data-frames
S_15_mean.columns = ['customer_ID', 'S_15_mean']
S_15_median.columns = ['customer_ID', 'S_15_median']

## Merging the resulting data-frames together
S_15_variable = S_15_mean.merge(S_15_median, how = 'left', on = 'customer_ID')

## Merging back into the complete train data-frame
train = train.merge(S_15_variable, how = 'left', on = 'customer_ID')

In [ ]:
## Grouping the data by customer_ID to obtain the mean and median S_27 variable values
S_27_mean = pd.DataFrame(train.groupby(['customer_ID'])['S_27'].mean()).reset_index(drop = False)
S_27_median = pd.DataFrame(train.groupby(['customer_ID'])['S_27'].median()).reset_index(drop = False)

## Cleaning the resulting data-frames
S_27_mean.columns = ['customer_ID', 'S_27_mean']
S_27_median.columns = ['customer_ID', 'S_27_median']

## Merging the resulting data-frames together
S_27_variable = S_27_mean.merge(S_27_median, how = 'left', on = 'customer_ID')

## Merging back into the complete train data-frame
train = train.merge(S_27_variable, how = 'left', on = 'customer_ID')

In [ ]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
## Using the train data-frame to engineer variable interactions

## Defining the input and target variables
X = train[['P_2', 'P_3', 'P_4', 'S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_9', 'S_11', 'S_12', 'S_13', 'S_15', 
           'S_16', 'S_17', 'S_18', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27']]
Y = train['target']

## Defining a list to store results
results = []

## Repeating process 100 times
for i in tqdm(range(0, 5)):
    
    ## Splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.8, stratify = Y)
    
    ## Building the model
    rf_md = RandomForestClassifier(max_depth = 3, n_estimators = 100).fit(X_train, Y_train)
    
    ## Extracting feature importance scores
    results.append(rf_md.feature_importances_)
    
## Changing results list to a dataframe
results = pd.DataFrame(results, columns = X.columns)

## Computing averages and sorting variables by importance
results = pd.DataFrame(results.apply(np.mean, axis = 0))
results = pd.DataFrame({'Feature': results.index, 'Importance': results[0].values}).sort_values(by = 'Importance', ascending = False)

results